In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Lobby as LobbyModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel
from app.models import LobbySeed as LobbySeedModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

Models = [
    BracketModel, 
    LobbyModel,
    TournamentModel, 
    MatchModel, 
    RoundModel, 
    UserModel, 
    LobbySeedModel
]

for Model in Models:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# log miguel in
url = 'http://127.0.0.1:5000/api/login'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"miguel logged in"}\n'

In [10]:
# log miguel out
url = 'http://127.0.0.1:5000/api/logout'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"logged out"}\n'

In [11]:
url = 'http://127.0.0.1:5000/api/users'
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })
    r = requests.post(url, data=payload, headers=headers)


In [12]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [13]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NDM3NjUyOCwiZXhwIjoxNTg0Mzc3MTI4fQ.eyJpZCI6MX0.baJ3AVWUZ_UPUTyVleMx4VL9Qt8LnyLyRgZZk-smE7p46xz1vxMWQbO-gpGsIQ6hhGd7w7c9enkFlmym6-OgWA'

In [14]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [15]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourneys"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Lobby Created': "Lobby created for tournament: miguel's tourneys"}

# Empty lobby 

In [16]:
q = \
"""
    select * from lobby_seeds where lobby_id == 1
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,user_id,lobby_id,seed


In [17]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourneys,None,1


In [18]:
q = \
"""
    select * from user
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$L5teuV0q$dfd5b0bfa1c8a037...,None,2020-03-16 16:35:27.693635
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$hcWyTFfl$3fa53d761b117e86...,None,2020-03-16 16:35:28.022663
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$qhUSCiU8$49b9e72c83e1ad11...,None,2020-03-16 16:35:28.111111
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$siFlE0pq$3ffc484617f2a050...,None,2020-03-16 16:35:28.198685
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$Ov2d7HyR$9190ad96da42316f...,None,2020-03-16 16:35:28.283038
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$g2rZGY3n$5624753e20ddd358...,None,2020-03-16 16:35:28.367357
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$TDI1ZGUO$05f364e7a78ae61f...,None,2020-03-16 16:35:28.452244
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$IZlGV1Hw$fdf11a71a486259a...,None,2020-03-16 16:35:28.539107
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$3wjlejxg$14c1236b5e20071c...,None,2020-03-16 16:35:28.622435


In [19]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NDM3NjUyOCwiZXhwIjoxNTg0Mzc3MTI4fQ.eyJpZCI6MX0.baJ3AVWUZ_UPUTyVleMx4VL9Qt8LnyLyRgZZk-smE7p46xz1vxMWQbO-gpGsIQ6hhGd7w7c9enkFlmym6-OgWA'

In [20]:
users = df.username.values
np.random.shuffle(users)
import random

# miguel adds users to his lobby

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

seeds = random.sample(range(1, len(users)+1), len(users)) 
roles = ["User" for u in users]
for u, s, r in zip(users, roles, seeds):
    print(f'adding {u}, {s}, {r}')
    payload = json.dumps({
        "username" : u,
        "role" : s,
        "seed" : r
    })
    r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
    print(json.loads(r.content))
    

adding Sunrisebanana, User, 8
{'Added': 'User Sunrisebanana to lobby 1'}
adding Ptolemy, User, 6
{'Added': 'User Ptolemy to lobby 1'}
adding Burnaby, User, 4
{'Added': 'User Burnaby to lobby 1'}
adding TPN, User, 1
{'Added': 'User TPN to lobby 1'}
adding Kevin, User, 2
{'Added': 'User Kevin to lobby 1'}
adding Vik, User, 5
{'Added': 'User Vik to lobby 1'}
adding AngryFalco, User, 3
{'Added': 'User AngryFalco to lobby 1'}
adding Spaceghost, User, 7
{'Added': 'User Spaceghost to lobby 1'}
adding miguel, User, 9
{'Added': 'User miguel to lobby 1'}


In [21]:
url = 'http://127.0.0.1:5000/api/lobby/1'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
json.loads(r.content)

{'id': 1, 'tournament_name': "miguel's tourneys"}

In [22]:
# miguel adds a guest user

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestb",
    "role" : "Guest",
    "seed" : 10
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
print(json.loads(r.content))

{'Added': 'Guest guestb to lobby 1'}


In [23]:
# log miguel out
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# TPN tries to add his own guests to miguel's lobby
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestx",
    "role" : "Guest",
    "seed" : 1
})
r = requests.post(url, data=payload, headers=headers, auth=('TPN', 'tpn'))
print(json.loads(r.content))

UserModel \
    .query \
    .filter_by(username='guesta', role='Guest') \
    .first() is None

{'Unauthorized': 'TPN cannot add entrants to this lobby'}


True

In [24]:
url = 'http://127.0.0.1:5000/api/lobby/1/entrants'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
content =  json.loads(r.content)
content

[{'user': 4, 'lobby': 1, 'id': 1, 'seed': 8},
 {'user': 5, 'lobby': 1, 'id': 2, 'seed': 6},
 {'user': 9, 'lobby': 1, 'id': 3, 'seed': 4},
 {'user': 2, 'lobby': 1, 'id': 4, 'seed': 1},
 {'user': 7, 'lobby': 1, 'id': 5, 'seed': 2},
 {'user': 6, 'lobby': 1, 'id': 6, 'seed': 5},
 {'user': 3, 'lobby': 1, 'id': 7, 'seed': 3},
 {'user': 8, 'lobby': 1, 'id': 8, 'seed': 7},
 {'user': 1, 'lobby': 1, 'id': 9, 'seed': 9},
 {'user': 10, 'lobby': 1, 'id': 10, 'seed': 10}]

In [25]:
pd.read_sql_query(f'select * from lobby', cnx)

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourneys,None,1


In [26]:
pd.read_sql_query(f'select * from lobby_seeds', cnx)

,id,user_id,lobby_id,seed
0,1,4,1,8
1,2,5,1,6
2,3,9,1,4
3,4,2,1,1
4,5,7,1,2
5,6,6,1,5
6,7,3,1,3
7,8,8,1,7
8,9,1,1,9
9,10,10,1,10


In [27]:
url = 'http://127.0.0.1:5000/api/lobby/1/create-tournament/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({})
r = requests.post(url, payload, headers=headers, auth=(tkn, 'unused'))
# r = requests.post(url, payload, headers=headers, auth=('TPN', 'tpn'))
r



<Response [200]>

In [28]:
r.content

b'{"Success":"Created tournament 1"}\n'

In [29]:
json.loads(r.content)

{'Success': 'Created tournament 1'}

In [30]:
# create the dataframe from a query

print('#'*68)
print()

print('Users\' table')
df = pd.read_sql_query(q, cnx)
# df

['tournament', 'bracket', 'bracket_entrants', 'round', 'match', 'lobby', 
'lobby_seeds']
    
pd.read_sql_query(f'select * from tournament', cnx)

####################################################################

Users' table


,id,n_entrants,name,organizer_id,finished
0,1,10,miguel's tourneys,1,0


In [31]:
pd.read_sql_query(f'select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [32]:
pd.read_sql_query(f'select * from bracket_entrants', cnx)

,user_id,bracket_id
0,2,1
1,7,1
2,3,1
3,9,1
4,6,1
5,5,1
6,8,1
7,4,1
8,1,1
9,10,1


In [33]:
pd.read_sql_query(f'select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [75]:
pd.read_sql_query(f'select * from match', cnx)

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,8440b873-52c7-423d-a5e9-b755a77f77d7,2.0,2.0,4.0,0.0,2.0,1,5.0,10.0
1,2,62ddd66b-60d9-4d0a-9b34-e56a7a54c323,7.0,2.0,8.0,0.0,7.0,1,5.0,10.0
2,3,f8a7ebca-e76e-4275-bec3-24d4af40cc1c,3.0,2.0,5.0,0.0,3.0,1,6.0,11.0
3,4,0ab27369-1f9f-4134-ad5a-2ea971698a53,9.0,2.0,6.0,0.0,9.0,1,6.0,11.0
4,5,7fd8551b-f72a-4dc3-a84e-873f8ee7952e,2.0,2.0,7.0,0.0,NaN,2,7.0,12.0
5,6,2956fe58-297c-480f-97bf-2be497cb2c6b,3.0,2.0,9.0,0.0,NaN,2,7.0,13.0
6,7,928f7951-0777-469e-ba54-50e3b24a1fcd,NaN,NaN,NaN,NaN,NaN,3,8.0,15.0
7,8,146a8aa1-cdfb-4b58-bbe3-85277b13f55e,NaN,NaN,NaN,NaN,NaN,4,9.0,9.0
8,9,7751782e-4c38-4263-a503-0fa2473269cb,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN
9,10,1bdedede-02c1-4c51-ae29-42a5e4329092,4.0,2.0,8.0,0.0,NaN,6,12.0,NaN


In [35]:
pd.read_sql_query(f'select * from lobby', cnx)

,id,tournament_name,tournament_id,to_id


In [36]:
pd.read_sql_query(f'select * from lobby_seeds', cnx)

,id,user_id,lobby_id,seed


In [37]:
URL_PREFIX = 'http://127.0.0.1:5000/'

In [76]:
def handle_progression(tournament, tkn):
    """Mimic Progression Handling of a Tournament
    
    Arguments:
        tournament {TournamentMode} -- a sqlAlchemy db model represnting
        a double elimination bracket tournament
        tkn {string} -- a unique authentication token
        used to ensure that only the tournament organizer enters scores in a 
        bracket 
    """
    
    for bracket in tournament.brackets:
        for round in bracket.rounds:
            for match in round.matches:
                print(match.user_1)
                if match.id == 8:
                    urlgf = \
                        f'{URL_PREFIX}api/match/{match.id}/report-match'
                    payloadgf = json.dumps({
                        "entrant1_score": 0,
                        "entrant2_score": 3,
                        "winner": match.user_2,
                        "loser": match.user_1
                    })
                    
                    print(payloadgf)
                    
                elif match.id == 9:
                    urlgfr = \
                        f'{URL_PREFIX}api/match/{match.id}/report-match'
                    payloadgfr = json.dumps({
                        "entrant1_score": 3,
                        "entrant2_score": 0,
                        "winner": match.user_1,
                        "loser": match.user_2
                    })
                    
                    print(payloadgfr)
                    
                else:
                    url = \
                        f'{URL_PREFIX}api/match/{match.id}/report-match'
                    payload = json.dumps({
                        "entrant1_score": 2,
                        "entrant2_score": 0,
                        "winner": match.user_1,
                        "loser": match.user_2
                    })
                    
                    print('pl', payload)
                    
                    r = requests.post(
                        url, data=payload, headers=headers, 
                        auth=(tkn, 'unused')
                    )
                print(json.loads(r.content))
                print('-'*68)
    requests.post(urlgf, data=payloadgf, headers=headers, auth=(tkn, 'unused'))
    requests.post(urlgfr, data=payloadgfr, headers=headers, auth=(tkn, 'unused'))

In [66]:
q = 'select * from tournament'

# posted a user
df = pd.read_sql_query(q, cnx)
tid = df.head(1)['id'].values[0]
df

,id,n_entrants,name,organizer_id,finished
0,1,10,miguel's tourneys,1,0


In [40]:
tournament = TournamentModel.query.filter_by(id=int(tid)).first_or_404()

In [73]:
q = 'select user_1 from match where id == 5'

# posted a user
df = pd.read_sql_query(q, cnx)
df

,user_1
0,2


In [47]:
# get TPN's token / log TPN in
url = f'{URL_PREFIX}api/token'
r = requests.get(url, auth=('TPN', 'tpn'))
c_tkn = json.loads(r.content)['token']

# TPN can't enter scores
handle_progression(tournament, c_tkn)

{"entrant1_score": 2, "entrant2_score": 0, "winner": 2, "loser": 4}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": 7, "loser": 8}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": 3, "loser": 5}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": 9, "loser": 6}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": null, "loser": null}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": null, "loser": null}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 2, "entrant2_score": 0, "winner": null, "loser": null}
{'Access Denied': 'Only Tournament Organizers can report matches'}
{"entrant1_score": 0, "entrant2_score

In [77]:

tournament = TournamentModel.query.filter_by(id=int(tid)).first_or_404()
url = f'{URL_PREFIX}api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']

# TO (miguel) can enter scores
handle_progression(tournament, tkn)

2
pl {"entrant1_score": 2, "entrant2_score": 0, "winner": 2, "loser": 4}
{'id': 1, 'loser_to': 10, 'match_winner': 2, 'round': 1, 'u1': 2, 'u2': 4, 'user_1_score': 2, 'user_2_score': 0, 'uuid': '8440b873-52c7-423d-a5e9-b755a77f77d7', 'winner_to': 5}
--------------------------------------------------------------------
7
pl {"entrant1_score": 2, "entrant2_score": 0, "winner": 7, "loser": 8}
{'id': 2, 'loser_to': 10, 'match_winner': 7, 'round': 1, 'u1': 7, 'u2': 8, 'user_1_score': 2, 'user_2_score': 0, 'uuid': '62ddd66b-60d9-4d0a-9b34-e56a7a54c323', 'winner_to': 5}
--------------------------------------------------------------------
3
pl {"entrant1_score": 2, "entrant2_score": 0, "winner": 3, "loser": 5}
{'id': 3, 'loser_to': 11, 'match_winner': 3, 'round': 1, 'u1': 3, 'u2': 5, 'user_1_score': 2, 'user_2_score': 0, 'uuid': 'f8a7ebca-e76e-4275-bec3-24d4af40cc1c', 'winner_to': 6}
--------------------------------------------------------------------
9
pl {"entrant1_score": 2, "entrant2_score"